<a href="https://colab.research.google.com/github/ImPeekaboo/mytools/blob/main/peeklab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

####**Setup**

In [ ]:
# ============================= FORM ============================= #
#@markdown <font size=4>**Install rclone**</font>
build_version = "stable" #@param ["stable", "beta"]
automatically_clear_cell_output = True  # @param{type: "boolean"}
# ================================================================ #

import os, urllib.request, time
from IPython.display import clear_output

if not os.path.exists("/root/.ipython/peeklab.py"):
    urllib.request.urlretrieve("https://raw.githubusercontent.com/ImPeekaboo/mytools/main/source/peeklab.py", "/root/.ipython/peeklab.py")

from peeklab import installRclone, installRcloneB

if build_version == "stable":
    installRclone()
else:
    installRcloneB()

try:
    os.makedirs("/root/.config/rclone", exist_ok=True)
except OSError as error:
    pass

if automatically_clear_cell_output is True:
    time.sleep(2)
    clear_output()

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=4>**Upload Configuration File**</font>
# @markdown <p>If you already have an rclone configuration file, you can upload it by running this cell.</p>
# ================================================================ #

from google.colab import files
from IPython.display import clear_output
import shutil
import os
import time

uploaded = files.upload()

destination_directory = '/root/.config/rclone'

for filename in uploaded.keys():
    shutil.move(filename, destination_directory + '/' + filename)

clear_output()
print(filename, 'success uploaded')
time.sleep(2)
clear_output()

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=4>**Mount Google Drive**</font>
# @markdown <p>Run this if you don't want to upload rclone config, but you still need run ***Install rclone*** above to use rclone transfer to transfer archive-result to your google drive.</p>
# ================================================================ #

import os
delete_path = '/content/sample_data'
if os.path.exists(delete_path):
    !rm -rf '/content/sample_data'

from google.colab import drive
drive.mount('/content/drive')

####**MEGA**

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=4>**MEGA Downloader**</font>
URL = "https://mega.nz/folder/dD8xlRaR#LhgrDHLyyZZOydA7natzdg/file/cP9B0C4I" #@param {type:"string"}
OUTPUT_PATH = "/content/downloads" #@param {type:"string"}
# ================================================================ #

import sys, os, urllib.request, time, subprocess, contextlib
from IPython.display import clear_output, update_display

if not os.path.exists("/root/.ipython/peeklab.py"):
    urllib.request.urlretrieve("https://raw.githubusercontent.com/ImPeekaboo/mytools/main/source/peeklab.py", "/root/.ipython/peeklab.py")

from peeklab import installMEGA

if not os.path.exists(OUTPUT_PATH):
  os.makedirs(OUTPUT_PATH, exist_ok=True)

# Installing MEGAcmd
if not os.path.exists("/usr/bin/mega-cmd"):
    installingMEGA()

# Unix, Windows and old Macintosh end-of-line
newlines = ['\n', '\r\n', '\r']

def unbuffered(proc, stream='stdout'):
    stream = getattr(proc, stream)
    with contextlib.closing(stream):
        while True:
            out = []
            last = stream.read(1)
            # Don't loop forever
            if last == '' and proc.poll() is not None:
                break
            while last not in newlines:
                # Don't loop forever
                if last == '' and proc.poll() is not None:
                    break
                out.append(last)
                last = stream.read(1)
            out = ''.join(out)
            yield out

def transfare():
    import codecs
    decoder = codecs.getincrementaldecoder("UTF-8")()
    cmd = ["mega-get", URL, OUTPUT_PATH]
    proc = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        # Make all end-of-lines '\n'
        universal_newlines=True,
    )
    for line in unbuffered(proc):
        #clear_output(wait=True)
        print(line)
transfare()

####**Libtorrent**

In [ ]:

# ============================= FORM ============================= #
# @markdown <font size=4>**libtorrent for torrent file**</font>
# @markdown <p>Run this cell to use libtorrent with torrent file.</p>
# ================================================================ #

from IPython.display import clear_output
import os, time, importlib, urllib.request, sys

if not os.path.exists("/root/.ipython/peeklab.py"):
    urllib.request.urlretrieve("https://raw.githubusercontent.com/ImPeekaboo/mytools/main/source/peeklab.py", "/root/.ipython/peeklab.py")

from peeklab import installLT

if os.path.exists("/content/downloads"):
  pass
else:
  os.makedirs("/content/downloads")

if importlib.util.find_spec("libtorrent") is None:
    installLT()

import libtorrent as lt
from google.colab import files

ses = lt.session({'listen_interfaces': '0.0.0.0:6881'})

uploaded = files.upload()
uploaded_file_path = list(uploaded.keys())[0]
clear_output()

info = lt.torrent_info(uploaded_file_path)
h = ses.add_torrent({'ti': info, 'save_path': '/content/downloads'})
s = h.status()

print('start downloading', s.name)

!rm "{uploaded_file_path}"

while (not s.is_seeding):
    s = h.status()

    print('\r%.2f%% complete (down: %.1f kB/s up: %.1f kB/s peers: %d) %s' % (
        s.progress * 100, s.download_rate / 1000, s.upload_rate / 1000,
        s.num_peers, s.state), end=' ')


# this optional, you can include this too
#    alerts = ses.pop_alerts()
#    for a in alerts:
#        if a.category() & lt.alert.category_t.error_notification:
#            print(a)

    sys.stdout.flush()

    time.sleep(1)

print(h.status().name, 'complete')

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=4>**libtorrent for magnet link**</font>
# @markdown <p>Run this cell to use libtorrent with magnet link.</p>
# ================================================================ #

from IPython.display import clear_output
import os, time, importlib, urllib.request

if not os.path.exists("/root/.ipython/peeklab.py"):
    urllib.request.urlretrieve("https://raw.githubusercontent.com/ImPeekaboo/mytools/main/source/peeklab.py", "/root/.ipython/peeklab.py")

from peeklab import installLT

if os.path.exists("/content/downloads"):
  pass
else:
  os.makedirs("/content/downloads")

if importlib.util.find_spec("libtorrent") is None:
    installLT()

import libtorrent as lt
import sys, urllib.parse

ses = lt.session({'listen_interfaces': '0.0.0.0:6881'})

magnet_link = input('Paste Magnet Link here:\n')
clear_output()

params = lt.add_torrent_params()
params.save_path = '/content/downloads'

info = lt.parse_magnet_uri(magnet_link)
info_hash = info.info_hashes
params.info_hashes = info_hash

h = ses.add_torrent(params)
s = h.status()

parsed_magnet_link = urllib.parse.urlparse(magnet_link)
torrent_name = urllib.parse.unquote(parsed_magnet_link.query.split('&dn=')[1].split('&')[0])

print('start downloading', torrent_name)

while (not s.is_seeding):
    s = h.status()

    print('\r%.2f%% complete (down: %.1f kB/s up: %.1f kB/s peers: %d) %s' % (
        s.progress * 100, s.download_rate / 1000, s.upload_rate / 1000,
        s.num_peers, s.state), end=' ')

#    alerts = ses.pop_alerts()
#    for a in alerts:
#        if a.category() & lt.alert.category_t.error_notification:
#            print(a)

    sys.stdout.flush()

    time.sleep(1)

print(h.status().name, 'complete')

####**Gofile-dl**

In [ ]:
#@markdown <font size=4>**Gofile Downloader**</font>
link = "https://gofile.io/d/nw79ZN" #@param{type:"string"}
output_path = "/content/downloads" #@param{type:"string"}

import os, urllib.request
from IPython.display import clear_output

if not os.path.exists("/root/.ipython/peeklab.py"):
    urllib.request.urlretrieve("https://raw.githubusercontent.com/ImPeekaboo/mytools/main/source/peeklab.py", "/root/.ipython/peeklab.py")

from peeklab import installGF

if not os.path.exists("/usr/local/bin/gofile"):
    installGF()

if not os.path.exists(output_path):
   os.makedirs(output_path)

os.environ['GF_DOWNLOADDIR'] = output_path

!gofile "{link}"

In [ ]:
#@markdown <font size=4>**cyberdrop-dl**</font>
link = "https://uptobox.com/mmdz5rdfpdsy" #@param{type:"string"}
output = "/content/downloads" #@param{type:"string"}
discord_notif = False #@param {type:"boolean"}
hide_progress = True #@param {type:"boolean"}

import importlib, os, urllib.request
from IPython.display import clear_output

if not os.path.exists("/root/.ipython/peeklab.py"):
    urllib.request.urlretrieve("https://raw.githubusercontent.com/ImPeekaboo/mytools/main/source/peeklab.py", "/root/.ipython/peeklab.py")

from peeklab import installCD

if importlib.util.find_spec("cyberdrop_dl") is None:
    installCD()

command = f"cyberdrop-dl --refresh-rate 100 --hide-new-progress -o \"{output}\" \"{link}\""
if hide_progress is True:
    command = command.replace('--hide-new-progress', '--hide-overall-progress --hide-forum-progress --hide-thread-progress --hide-domain-progress --hide-album-progress --hide-file-progress')
!{command}

if discord_notif is True:
    import requests
    import json

    webhook_url = 'https://discord.com/api/webhooks/1120796600676130816/1KJlFD2SFGrAKB2sA9E07nSa5IhGvofZJGvUb_-BF7N1AUXXBJpy6KSw4k7njZNAm0Wv'

    message = ('@everyone Rclone Done!')

    payload = {
        'content': message
    }

    response = requests.post(webhook_url, data=json.dumps(payload), headers={'Content-Type': 'application/json'})

    if response.status_code == 204:
        print('Message sent to Discord!')
    else:
        print('Failed send message to Discord. Response status code:', response.status_code)

####**Youtube Archive**

In [ ]:
# ============================= FORM ============================= #
# @markdown <p>Run this first before you use <i><b>Archive livestreaming</b></i> and <i><b>Archive processed video/VOD</b></i>.</p>
# ================================================================ #

from IPython.display import clear_output
import os, urllib.request
import time

if not os.path.exists("/root/.ipython/peeklab.py"):
    urllib.request.urlretrieve("https://raw.githubusercontent.com/ImPeekaboo/mytools/main/source/peeklab.py", "/root/.ipython/peeklab.py")

from peeklab import setupYTA

setupYTA()

# you can manually build yta with go with this
# !wget https://go.dev/dl/go1.20.5.linux-amd64.tar.gz
# !tar -xf go1.20.5.linux-amd64.tar.gz
# !sudo mv go /usr/local
# !rm go1.20.5.linux-amd64.tar.gz
# os.environ['PATH'] += ":/usr/local/go/bin"
# !git clone https://github.com/ImPeekaboo/ytarchive
# !cd ytarchive && go build -o /usr/local/bin/ytarchive
# !rm -rf ytarchive

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=4>**Upload your cookies.txt here**</font>
# @markdown <p>If you want to archive membership content, you need upload your cookies file, before you upload it you have to name it become ***cookies.txt***.</p>
# ================================================================ #

from google.colab import files
from IPython.display import clear_output
import time

uploaded = files.upload()
clear_output()
for filename in uploaded.keys():
    print(filename, 'success uploaded')
time.sleep(2)
clear_output()

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=4>**Archive livestreaming**</font>
# @markdown <p>Fill ***youtube_id*** with 11 digits youtube video link id.</p>
# ================================================================ #

youtube_id = "5ZMpvIFnaqs" #@param {type:"string"}

import os
import threading

cookies_path = '/content/cookies.txt'

def run_command_1():
    command_1 = "ytarchive -v -w -r 1 -o '/content/archive-result/[%(upload_date)s] %(title)s [%(channel)s] (%(id)s)/[%(upload_date)s] %(title)s [%(channel)s] (%(id)s)' -t --add-metadata --write-description --write-thumbnail --threads 4 --merge 'https://www.youtube.com/watch?v={}' best".format(youtube_id)
    if os.path.isfile(cookies_path):
        command_1 = command_1.replace("ytarchive", "ytarchive --cookies cookies.txt")
    !{command_1}

def run_command_2():
    command_2 = "yt-dlp -v -c -o '/content/archive-result/[%(release_date,upload_date)s] %(title)s [%(channel)s] (%(id)s)/[%(release_date,upload_date)s] %(title)s [%(channel)s] (%(id)s)' --wait-for-video 1 --live-from-start --concurrent-fragments 20 --write-info-json --clean-infojson --write-comments --write-subs --sub-lang live_chat --sub-format json --skip-download 'https://www.youtube.com/watch?v={}'".format(youtube_id)
    if os.path.isfile(cookies_path):
        command_2 = command_2.replace("yt-dlp", "yt-dlp --cookies cookies.txt")
    !{command_2}

thread_1 = threading.Thread(target=run_command_1)
thread_2 = threading.Thread(target=run_command_2)

thread_1.start()
thread_2.start()

thread_1.join()
thread_2.join()

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=4>**Archive processed video/VOD**</font>
# @markdown <p>Fill ***youtube_id*** with 11 digits youtube video link id.</p>
# ================================================================ #

youtube_id = "FL-0l9_28yM" #@param {type:"string"}

import os

cookies_path = '/content/cookies.txt'

def run_command_1():
    command_1 = "yt-dlp -v -c -o '/content/archive-result/[%(release_date,upload_date)s] %(title)s [%(channel)s] (%(id)s)/[%(release_date,upload_date)s] %(title)s [%(channel)s] (%(id)s)' --add-metadata --concurrent-fragments 20 --merge-output-format mp4 --embed-thumbnail --embed-metadata --write-info-json --clean-infojson --write-comments --write-subs --sub-lang live_chat --sub-format json --write-description --write-thumbnail --convert-thumbnails jpg 'https://www.youtube.com/watch?v={}'".format(youtube_id)
    if os.path.isfile(cookies_path):
        command_1 = command_1.replace("yt-dlp", "yt-dlp --cookies cookies.txt")
    !{command_1}
run_command_1()

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=4>**Archive Youtube Tab or Playlist**</font>
# @markdown <p>Fill ***link*** with youtube tab or playlist link. This cell can download 6 index configuration at once with multi thread and you can fill ***index*** with number to configure which index on playlist or youtube tab that you want to download. ***short_filename*** is not complete filename format to make less error when downloading, uncheck if you want complete filename.</p>
link = "https://youtube.com/watch?v=S-ZtuuItlGI" #@param {type:"string"}
index_1 = "" #@param {type:"string"}
index_2 = "" #@param {type:"string"}
index_3 = "" #@param {type:"string"}
index_4 = "" #@param {type:"string"}
index_5 = "" #@param {type:"string"}
index_6 = "" #@param {type:"string"}
short_filename = False  # @param {type: "boolean"}
# ================================================================ #

import os
import threading

cookies_path = '/content/cookies.txt'

def run_command_1():
    if index_1:
        command_1 = "yt-dlp -v -c -o '/content/archive-result/[%(release_date,upload_date)s] %(title)s [%(channel)s] (%(id)s)/[%(release_date,upload_date)s] %(title)s [%(channel)s] (%(id)s)' --add-metadata --concurrent-fragments 20 --merge-output-format mp4 --mark-watched --embed-thumbnail --embed-metadata --write-info-json --clean-infojson --write-comments --write-subs --sub-lang live_chat --sub-format json --write-description --write-thumbnail --convert-thumbnails jpg '{}'".format(link)
        if os.path.isfile(cookies_path):
            command_1 = command_1.replace("yt-dlp", "yt-dlp --cookies cookies.txt")
        command_1 = command_1.replace("--add-metadata", f"--add-metadata -I {index_1}")
        if short_filename is True:
            command_1 = command_1.replace("[%(release_date,upload_date)s] %(title)s [%(channel)s] (%(id)s)/[%(release_date,upload_date)s] %(title)s [%(channel)s] (%(id)s)", "[%(release_date,upload_date)s] (%(id)s)/[%(release_date,upload_date)s] (%(id)s)")
        !{command_1}
    else:
        pass

def run_command_2():
    if index_2:
        command_2 = "yt-dlp -v -c -o '/content/archive-result/[%(release_date,upload_date)s] %(title)s [%(channel)s] (%(id)s)/[%(release_date,upload_date)s] %(title)s [%(channel)s] (%(id)s)' --add-metadata --concurrent-fragments 20 --merge-output-format mp4 --mark-watched --embed-thumbnail --embed-metadata --write-info-json --clean-infojson --write-comments --write-subs --sub-lang live_chat --sub-format json --write-description --write-thumbnail --convert-thumbnails jpg '{}'".format(link)
        if os.path.isfile(cookies_path):
            command_2 = command_2.replace("yt-dlp", "yt-dlp --cookies cookies.txt")
        command_2 = command_2.replace("--add-metadata", f"--add-metadata -I {index_2}")
        if short_filename is True:
            command_2 = command_2.replace("[%(release_date,upload_date)s] %(title)s [%(channel)s] (%(id)s)/[%(release_date,upload_date)s] %(title)s [%(channel)s] (%(id)s)", "[%(release_date,upload_date)s] (%(id)s)/[%(release_date,upload_date)s] (%(id)s)")
        !{command_2}
    else:
        pass

def run_command_3():
    if index_3:
        command_3 = "yt-dlp -v -c -o '/content/archive-result/[%(release_date,upload_date)s] %(title)s [%(channel)s] (%(id)s)/[%(release_date,upload_date)s] %(title)s [%(channel)s] (%(id)s)' --add-metadata --concurrent-fragments 20 --merge-output-format mp4 --mark-watched --embed-thumbnail --embed-metadata --write-info-json --clean-infojson --write-comments --write-subs --sub-lang live_chat --sub-format json --write-description --write-thumbnail --convert-thumbnails jpg '{}'".format(link)
        if os.path.isfile(cookies_path):
            command_3 = command_3.replace("yt-dlp", "yt-dlp --cookies cookies.txt")
        command_3 = command_3.replace("--add-metadata", f"--add-metadata -I {index_3}")
        if short_filename is True:
            command_3 = command_3.replace("[%(release_date,upload_date)s] %(title)s [%(channel)s] (%(id)s)/[%(release_date,upload_date)s] %(title)s [%(channel)s] (%(id)s)", "[%(release_date,upload_date)s] (%(id)s)/[%(release_date,upload_date)s] (%(id)s)")
        !{command_3}
    else:
        pass

def run_command_4():
    if index_4:
        command_4 = "yt-dlp -v -c -o '/content/archive-result/[%(release_date,upload_date)s] %(title)s [%(channel)s] (%(id)s)/[%(release_date,upload_date)s] %(title)s [%(channel)s] (%(id)s)' --add-metadata --concurrent-fragments 20 --merge-output-format mp4 --mark-watched --embed-thumbnail --embed-metadata --write-info-json --clean-infojson --write-comments --write-subs --sub-lang live_chat --sub-format json --write-description --write-thumbnail --convert-thumbnails jpg '{}'".format(link)
        if os.path.isfile(cookies_path):
            command_4 = command_4.replace("yt-dlp", "yt-dlp --cookies cookies.txt")
        command_4 = command_4.replace("--add-metadata", f"--add-metadata -I {index_4}")
        if short_filename is True:
            command_4 = command_4.replace("[%(release_date,upload_date)s] %(title)s [%(channel)s] (%(id)s)/[%(release_date,upload_date)s] %(title)s [%(channel)s] (%(id)s)", "[%(release_date,upload_date)s] (%(id)s)/[%(release_date,upload_date)s] (%(id)s)")
        !{command_4}
    else:
        pass

def run_command_5():
    if index_5:
        command_5 = "yt-dlp -v -c -o '/content/archive-result/[%(release_date,upload_date)s] %(title)s [%(channel)s] (%(id)s)/[%(release_date,upload_date)s] %(title)s [%(channel)s] (%(id)s)' --add-metadata --concurrent-fragments 20 --merge-output-format mp4 --mark-watched --embed-thumbnail --embed-metadata --write-info-json --clean-infojson --write-comments --write-subs --sub-lang live_chat --sub-format json --write-description --write-thumbnail --convert-thumbnails jpg '{}'".format(link)
        if os.path.isfile(cookies_path):
            command_5 = command_5.replace("yt-dlp", "yt-dlp --cookies cookies.txt")
        command_5 = command_5.replace("--add-metadata", f"--add-metadata -I {index_5}")
        if short_filename is True:
            command_5 = command_5.replace("[%(release_date,upload_date)s] %(title)s [%(channel)s] (%(id)s)/[%(release_date,upload_date)s] %(title)s [%(channel)s] (%(id)s)", "[%(release_date,upload_date)s] (%(id)s)/[%(release_date,upload_date)s] (%(id)s)")
        !{command_5}
    else:
        pass

def run_command_6():
    if index_6:
        command_6 = "yt-dlp -v -c -o '/content/archive-result/[%(release_date,upload_date)s] %(title)s [%(channel)s] (%(id)s)/[%(release_date,upload_date)s] %(title)s [%(channel)s] (%(id)s)' --add-metadata --concurrent-fragments 20 --merge-output-format mp4 --mark-watched --embed-thumbnail --embed-metadata --write-info-json --clean-infojson --write-comments --write-subs --sub-lang live_chat --sub-format json --write-description --write-thumbnail --convert-thumbnails jpg '{}'".format(link)
        if os.path.isfile(cookies_path):
            command_6 = command_6.replace("yt-dlp", "yt-dlp --cookies cookies.txt")
        command_6 = command_6.replace("--add-metadata", f"--add-metadata -I {index_6}")
        if short_filename is True:
            command_6 = command_6.replace("[%(release_date,upload_date)s] %(title)s [%(channel)s] (%(id)s)/[%(release_date,upload_date)s] %(title)s [%(channel)s] (%(id)s)", "[%(release_date,upload_date)s] (%(id)s)/[%(release_date,upload_date)s] (%(id)s)")
        !{command_6}
    else:
        pass

thread_1 = threading.Thread(target=run_command_1)
thread_2 = threading.Thread(target=run_command_2)
thread_3 = threading.Thread(target=run_command_3)
thread_4 = threading.Thread(target=run_command_4)
thread_5 = threading.Thread(target=run_command_5)
thread_6 = threading.Thread(target=run_command_6)

thread_1.start()
thread_2.start()
thread_3.start()
thread_4.start()
thread_5.start()
thread_6.start()

thread_1.join()
thread_2.join()
thread_3.join()
thread_4.join()
thread_5.join()
thread_6.join()

#message
import requests
import json

webhook_url = 'https://discord.com/api/webhooks/1120796600676130816/1KJlFD2SFGrAKB2sA9E07nSa5IhGvofZJGvUb_-BF7N1AUXXBJpy6KSw4k7njZNAm0Wv'

message = ('@everyone ytarchive Done!')

payload = {
    'content': message
}

response = requests.post(webhook_url, data=json.dumps(payload), headers={'Content-Type': 'application/json'})

if response.status_code == 204:
    print('Pesan terkirim ke Discord!')
else:
    print('Gagal mengirim pesan ke Discord. Kode status respons:', response.status_code)

####**Rclone Transfer**

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=4>**Transfer Files with Rclone**</font>
# @markdown <p>Use ***copy*** mode for files from your path and ***copyurl*** mode for direct link download, fill ***source*** with link or patch source, ***dest*** with your destiny path.</p>
mode = "copy" #@param ["copy", "copyurl"]
source = "/content/archive-result" #@param {type:"string"}
dest = "Onedrive:Public2" #@param {type:"string"}
discord_notif = False #@param {type:"boolean"}
# ================================================================ #

# command
if mode == "copyurl":
    command = f"rclone {mode} -v --stats 1 --stats-one-line --checkers=16 --onedrive-encoding \"Slash\" --create-empty-src-dirs -a \"{source}\" \"{dest}\" --user-agent 'ISV|rclone.org|rclone/v1.63.1'"
else:
    command = f"rclone {mode} -v --stats 1 --stats-one-line --checkers=16 --onedrive-encoding \"Slash\" --create-empty-src-dirs \"{source}\" \"{dest}\" --user-agent 'ISV|rclone.org|rclone/v1.63.1'"

!{command}

#notif for discord
if discord_notif is True:
    import requests
    import json

    webhook_url = 'https://discord.com/api/webhooks/1120796600676130816/1KJlFD2SFGrAKB2sA9E07nSa5IhGvofZJGvUb_-BF7N1AUXXBJpy6KSw4k7njZNAm0Wv'

    message = ('@everyone Rclone Done!')

    payload = {
        'content': message
    }

    response = requests.post(webhook_url, data=json.dumps(payload), headers={'Content-Type': 'application/json'})

    if response.status_code == 204:
        print('Message sent to ke Discord!')
    else:
        print('Failed send message to Discord. Response status code:', response.status_code)

In [ ]:
#@markdown **safe to interrupt**
youtube_id = "eE91ZWYi6rM" #@param {type:"string"}

import os
import threading

cookies_path = '/content/cookies.txt'

def run_command_1(stop_event_1):
    command_1 = "ytarchive -v -w -r 1 -o '/content/archive-result/[%(upload_date)s] %(title)s [%(channel)s] (%(id)s)/[%(upload_date)s] %(title)s [%(channel)s] (%(id)s)' -t --add-metadata --write-description --write-thumbnail --threads 4 --merge 'https://www.youtube.com/watch?v={}' best".format(youtube_id)
    if os.path.isfile(cookies_path):
        command_1 = command_1.replace("ytarchive", "ytarchive --cookies cookies.txt")
    !{command_1}
    stop_event_1.set()

def run_command_2(stop_event_2):
    command_2 = "yt-dlp -v -c -o '/content/archive-result/[%(release_date,upload_date)s] %(title)s [%(channel)s] (%(id)s)/[%(release_date,upload_date)s] %(title)s [%(channel)s] (%(id)s)' --wait-for-video 1 --live-from-start --concurrent-fragments 20 --write-info-json --clean-infojson --write-comments --write-subs --sub-lang live_chat --sub-format json --skip-download 'https://www.youtube.com/watch?v={}'".format(youtube_id)
    if os.path.isfile(cookies_path):
        command_2 = command_2.replace("yt-dlp", "yt-dlp --cookies cookies.txt")
    !{command_2}
    stop_event_2.set()

# Inisialisasi objek Event
stop_event_1 = threading.Event()
stop_event_2 = threading.Event()

# Jalankan thread
thread_1 = threading.Thread(target=run_command_1, args=(stop_event_1,))
thread_2 = threading.Thread(target=run_command_2, args=(stop_event_2,))

thread_1.start()
thread_2.start()

# Menghentikan thread dengan aman
stop_event_1.set()
stop_event_2.set()

try:
    # Sel Colab akan berhenti setelah beberapa waktu, sehingga kita menunggu thread selesai tanpa timeout
    thread_1.join()
    thread_2.join()
except KeyboardInterrupt:
    # Jika sel Colab diinterupsi, atur Event agar thread berhenti
    stop_event_1.set()
    stop_event_2.set()
    # Tunggu thread berhenti dengan join setelah Event diatur
    thread_1.join()
    thread_2.join()

####**test**

In [ ]:
!rm -rf "/content/archive-result/[20230730] 【#ホロサマ歌枠リレー】SUMMER IS HERE, LET'S SING WITH ME!!!　夏が来た！！！いっぱい歌おう！【Pavolia Reine⧸hololiveID 2nd gen】 [Pavolia Reine Ch. hololive-ID] (FL-0l9_28yM)"

In [ ]:
!wget https://github.com/Aerion/uptobox-dl/releases/download/v1.4.2/uptobox-dl-v1.4.2-linux-x64.tar.gz
!tar -xf uptobox-dl-v1.4.2-linux-x64.tar.gz
!mv uptobox-dl-v1.4.2-linux-x64/uptobox-dl /usr/local/bin/ && chmod +x /usr/local/bin/uptobox-dl
!rm uptobox-dl-v1.4.2-linux-x64.tar.gz && rm -rf uptobox-dl-v1.4.2-linux-x64

In [ ]:
0ac8ae3f7d7856d4d855c227f97c90d8cumxd

In [ ]:
!uptobox-dl -t 0ac8ae3f7d7856d4d855c227f97c90d8cumxd linkhere

In [ ]:

import os

def get_file_size_auto(file_path):
    if os.path.isfile(file_path):
        size_in_bytes = os.path.getsize(file_path)
        size_name = ('bytes', 'KB', 'MB', 'GB')
        size_idx = 0
        while size_in_bytes > 1024 and size_idx < len(size_name) - 1:
            size_in_bytes /= 1024.0
            size_idx += 1
        return f"{size_in_bytes:.2f} {size_name[size_idx]}"
    else:
        return "File tidak ditemukan."

file_path = "/content/downloads/0ed48563622b755819cd012bbd244159387a2be3/【耳舐め✨ASMR】バブみ感じる爆乳J●のママ♡年下にたっぷり甘やかされながら授乳プレイetc...♡【KU100-Ear licking】.mkv"
file_size_auto = get_file_size_auto(file_path)

print(f"Ukuran file '{file_path}' adalah {file_size_auto}.")